In [ ]:
from litellm import completion
from typing import List, Dict, Any, Optional
import jinja2
import json
from abc import ABC, abstractmethod
from tavily import TavilyClient
import os


# Objective: Create a conversation object that can be used to send messages to the model and get responses and tools
class Conversation:
    def __init__(self, model: str = "gemini/gemini-2.0-flash"):
        self.model = model
        self.messages = []

    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

    def get_response(self, tool_choice: str = "auto", tools: Optional[List] = None):
        if tools is None:
            tool_choice = None
        return completion(
            model=self.model,
            messages=self.messages,
            tools=tools,
            tool_choice=tool_choice,
        )


# Updated abstract Tool class that enforces `name` and `terminating` and accepts extra kwargs.
class Tool(ABC):
    def __init__(self, name: str, terminating: bool = False):
        self.name = name
        self.terminating = terminating

    @abstractmethod
    def run(self, conversation: Conversation, **kwargs) -> str:
        pass

    @abstractmethod
    def get_config(self) -> Dict[str, Any]:
        pass


# Reasoning tool (non‐terminating)
class Reasoning(Tool):
    def __init__(self):
        super().__init__(name="reasoning", terminating=False)

    def run(self, conversation: Conversation, **kwargs) -> str:
        with open("templates/reasoning.jinja2", "r") as file:
            reasoning_template = jinja2.Template(file.read()).render()
        conversation.add_message("system", reasoning_template)
        response = conversation.get_response()
        conversation.messages.pop()  # remove the reasoning message
        return response.choices[0].message.content

    def get_config(self) -> Dict[str, Any]:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "This tool provides reasoning for the given context.",
            },
        }


# Respond tool (terminating) – note the extra parameter `response_type`
class Respond(Tool):
    def __init__(self, response_types: List[str]):
        # Mark this tool as terminating since its response is meant to be final.
        super().__init__(name="respond", terminating=True)
        self.response_types = response_types

    def run(self, conversation: Conversation, response_type: str, **kwargs) -> str:
        with open(f"templates/{response_type}.jinja2", "r") as file:
            template_content = jinja2.Template(file.read()).render()
        conversation.add_message("system", template_content)
        response = conversation.get_response()
        conversation.messages.pop()
        return response.choices[0].message.content

    def get_config(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "This tool provides responses for the given user message.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "response_type": {
                            "type": "string",
                            "description": "The type of response to be generated.",
                            "enum": self.response_types,
                        },
                    },
                    "required": ["response_type"],
                },
            },
        }


# Search tool (non‐terminating)
class Search(Tool):
    def __init__(self):
        super().__init__(name="search", terminating=False)
        self.tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

    def run(self, conversation: Conversation, query: str, **kwargs) -> str:
        response = self.tavily_client.search(query)
        return response

    def get_config(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "This tool provides search results for the given query.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The query to search for.",
                        },
                    },
                    "required": ["query"],
                },
            },
        }


# An agent is a tool that can be used to interact with the model and get responses and use tools as required.
class Agent(Tool):
    def __init__(
        self,
        name: str,
        description: str,
        max_turns: int = 3,
        tools: List[Tool] = [],
        terminating_tools: List[Tool] = None,
        forced_terminating_tool: Tool = None,
    ):
        self._name = name
        self.description = description
        self.max_turns = max_turns
        self.available_tools = {tool.name: tool for tool in tools}
        self.terminating_tools = terminating_tools
        self.forced_terminating_tool = forced_terminating_tool

    @property
    def name(self) -> str:
        return self._name

    def run(self, conversation: Conversation) -> str:
        for turn in range(self.max_turns):
            tool, function_args = self.get_next_action(conversation)
            print(f"Using the {tool.name} tool with arguments {function_args}")
            response = tool.run(conversation, **function_args)
            if tool.name == self.terminating_tool.name:
                return response

            conversation.add_message(
                "user",
                f"The {tool.name} tool has been used and it responded with {response}",
            )
        return self.terminating_tool.run(self.conversation)

    def get_next_action(self, conversation: Conversation) -> Tool:
        response = conversation.get_response(
            tool_choice="required",
            tools=[tool.get_config() for tool in self.available_tools.values()],
        )
        tool_calls = response.choices[0].message.tool_calls
        function_name = tool_calls[0].function.name
        function_args = json.loads(tool_calls[0].function.arguments)
        return self.available_tools[function_name], function_args

    def get_config(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
            },
        }

Testing

In [14]:
with open("templates/system_message.jinja2", "r") as file:
    system_message = jinja2.Template(file.read()).render(tools="")

In [18]:


conversation = Conversation()
conversation.add_message("system", system_message)

respond = Respond(response_types=["respond", "clarification"])

agent = Agent(
    name="Autonomous market research agent",
    description="This agent is designed to help users with market research.",
    tools=[
        Reasoning(),
        respond,
        research_agent
    ],
    terminating_tool=respond,
)

conversation.add_message("user", "I want to research climate litigation")


response = agent.run(conversation)

print(response)

Using the research_agent tool with arguments {}
Using the search tool with arguments {'query': 'climate litigation'}
Using the reasoning tool with arguments {}
Using the respond tool with arguments {'response_type': 'write_report'}
Using the respond tool with arguments {'response_type': 'clarification'}
Okay, I see that you're interested in researching climate litigation. Based on your initial query and the search results, it seems like you want to understand:

*   **What climate litigation is:** The definition, types, and trends of climate change litigation.
*   **Key players:** Who is involved in these cases (plaintiffs and defendants).
*   **Goals of climate litigation:** What these cases aim to achieve (policy changes, compensation, etc.).
*   **Impact and challenges:** The successes, limitations, and overall impact of climate litigation.

Is that a correct assessment of what you're hoping to achieve with this research? Are there any specific aspects of climate litigation that you'